Домашнее задание №3
==

**НЕ изменяйте и не удаляйте ячейки, написанные с помощью markdown, только ячейки с кодом**

Справочные материалы по JSON и работе со списками и словарями:
* [Конспект лекций](https://sss.krasilnikov.spb.ru/class/4/uploader/file/1546944257_SSS_book.pdf), глава 5
* [Automate The Boring Stuff](https://automatetheboringstuff.com/), главы 4-5
* [DataCamp Tutorial](https://www.datacamp.com/community/tutorials/python-dictionary-tutorial) по приемам работы с dict

Вам необходимо будет загрузить для проверки в различные поля два файла:
1. этот ipynb файл
2. output.xls, полученный в последнем задании. Если какие-то столбцы не будут корректно посчитаны, экспортируйте файл, оставив в качестве значений 0 для всех наблюдений

* в переменной category1 у вас находится название набора данных
* при помощи запроса к [API портала открытых данных Москвы](https://apidata.mos.ru/Docs)  получите идентификатор набора данных category1 и сохраните его в переменной  id1

(**2 балла**)

In [ ]:
import json, numpy as np, pandas as pd, requests, urllib.parse as up

def api(caption=None, columns=None, dataset=None, params={'api_key': '0784bbb0a1fd86ac0a12753b881a2e86'}):
    if caption: params['$filter'] = f"Caption eq '{caption}'" 
    url_parts = ['https', 'apidata.mos.ru', 'v1/datasets', '', up.urlencode(params), ''] 
    if dataset: url_parts[2] += f'/{dataset}/rows' 
    response = requests.post(up.urlunparse(url_parts), json=columns) 
    return pd.io.json.json_normalize(json.loads(response.text)) 

category1 = "Кинотеатры"
id1 = api(category1, ['Id'])['Id'][0]
id1

495

* создайте датафрейм data_1
* загрузите значения следующих переменных по всем наблюдениям из набора данных из предыдущего задания, столбцы переименуйте в соответствии с указаными названиями:
 * global_id
 * seats (из Общее количество мест)
 * parent (из Вышестоящая организация)
* загрузите также координаты:
 * lat (широта)
 * lon (долгота)
 
(**3 балла**)

*Примечание: Если элемент "Общее количество мест" или "Вышестоящая организация" является списком, то в соответствующем столбце отображайте 0вой элемент.  
Если значение отсутствует - пустую строку*

In [ ]:
def process(df, columns, geodata='Cells.geoData.coordinates', phone=False):
    df = df.join(df[geodata].apply(lambda x: pd.Series(x[0], index=['lon', 'lat'])))
    df.rename(columns={f'Cells.{k}': v for k, v in columns.items()}, inplace=True)
    if phone: df['phone'] = df['phone'].apply(lambda x: x[0][phone][0][phone])
    return df[[c for c in columns.values() if c in df] + ['global_id', 'lon', 'lat']]
    
columns = {'TotalSeatsAmount': 'seats', 'ChiefOrg': 'parent', 'geoData': 'geoData'}
data_1 = api(columns=list(columns.keys()), dataset=id1)
data_1 = process(data_1, columns)
data_1.head()

,seats,parent,global_id,lon,lat
0,167,Государственное бюджетное учреждение культуры ...,477012208,37.571299,55.682807
1,633,Государственное бюджетное учреждение культуры ...,477012215,37.718515,55.685873
2,367,Государственное бюджетное учреждение культуры ...,477012217,37.802542,55.749856
3,120,Государственное бюджетное учреждение культуры ...,477012363,37.442906,55.846062
4,141,Государственное бюджетное учреждение культуры ...,477012415,37.676551,55.861865


_Аналогично заданию 1_
* в переменной category2 у вас находится название набора данных
* найдите идентификатор набора данных и сохраните его в переменной id2

(**1 балла**)

In [ ]:
category2 = "Отделы трудоустройства"
id2 = api(category2, ['Id'])['Id'][0]
id2

608

_Аналогично заданию 2_
* создайте датафрейм data_2
* загрузите значения следующих переменных по всем наблюдениям из набора данных из предыдущего задания, столбцы переименуйте в соответствии с указаными названиями:
 * global_id
 * phone (из Телефон руководителя)
 * fio (из ФИО руководителя)
* загрузите также координаты:
 * lat (широта)
 * lon (долгота)
 
(**1 балл**)

*Примечание: Если элемент "Телефон руководителя" или "ФИО руководителя" является списком, то в соответствующем столбце отображайте 0вой элемент.  
Если значение отсутствует - пустую строку*

In [ ]:
columns = {'OrgInfo': 'phone', 'ChiefName': 'fio', 'geoData': 'geoData'}
data_2 = api(columns=list(columns.keys()), dataset=id2)
data_2 = process(data_2, columns, phone='ChiefPhone')
data_2.head()

,phone,fio,global_id,lon,lat
0,(495) 675-82-28,Архипова Алла Гордеевна,641991307,37.540829,55.688949
1,(495) 675-82-28,Яворская Наталья Эдуардовна,641991311,37.656867,55.574731
2,(495) 675-82-28,Киняпина Анна Николаевна,641991328,37.665860,55.863685
3,(495) 675-82-28,Козлова Марина Николаевна,641991338,37.631856,55.804631
4,(495) 675-82-28,Уткина Лариса Алексеевна,641991342,37.655985,55.678208


* создайте датафрейм closest c указанными столбцами и заполните значениями
 * global_id1 - все идентификаторы из датафрейма data_1
 * global_id2 укажите идентификатор того объекта, который окажется ближе по прямой к объекту global_id1 из датафрейма data_1
 * dist - расстояние в метрах, рассчитанное по формуле

(**3 балла**)

_Справочно: Формула для расчета расстояния по координатам:  
S = 111,2×arccos(sin φ1 × sin φ2 + cos φ1 × cos φ2 × cos (L2-L1)),  
где S - расстояние, км;  
φ1 и L1 - широта и долгота точки 1 (для северной широты и восточной долготы со знаком плюс, для южной широты и западной долготы со знаком минус), градусы;  
φ2 и L2 - широта и долгота точки 2, градусы;  
константа 111,2 - средняя длина дуги в один градус на поверхности Земли, км._

_Справочно 2:
Для выполнения задания потребуется вычисление синуса, косинуса и арккосинуса. Краткий справочник доступен [здесь](https://pythonworld.ru/moduli/modul-math.html)_

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/d9bdd722860b46cf459e01fe0108fa1f45f13fd6)

In [ ]:
def haversine(a, b, D=12745.6):
    return D * np.arcsin(np.sqrt(np.dot([1, np.prod(np.cos(np.radians([a[0], b[0]])))], np.sin(np.radians(b - a) / 2) ** 2)))

def find_closest(a, df=data_2):
    df['dist'] = df[['lat', 'lon']].apply(lambda b: haversine(a, b), axis=1) * 1000
    return df[['global_id', 'dist']].sort_values('dist').iloc[0].astype(np.int)

closest = data_1[['global_id']].join(data_1[['lat', 'lon']].apply(find_closest, axis=1), lsuffix='1', rsuffix='2')
closest.head()

,global_id1,global_id2,dist
0,477012208,641991785,576
1,477012215,641991386,1229
2,477012217,641991691,1645
3,477012363,641991466,2014
4,477012415,641991328,697


* Экспортируйте данные из датафрейма closest в Excel файл output.xls

(**1 балл**)

In [ ]:
def download_xls(data, file_name='output.xls'):
    data.to_excel(file_name)
    try:
        from google.colab import files
        files.download(file_name)
    except Exception as error:
        print(error)

download_xls(closest)